## Imports

In [ ]:
import copy
import itertools
import logging
import math
from functools import partial
from pathlib import Path
from typing import Dict
import json

import hydra
import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import omegaconf
import pytorch_lightning
import seaborn as sns
import torch  # noqa
import wandb
from hydra.utils import instantiate
from matplotlib import tri
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from omegaconf import DictConfig
from pytorch_lightning import LightningModule
from scipy.stats import qmc
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler
from tqdm import tqdm

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import seed_index_everything
from nn_core.model_logging import NNLogger

import ccmm  # noqa
from ccmm.matching.utils import (
    apply_permutation_to_statedict,
    get_all_symbols_combinations,
    load_permutations,
    perm_indices_to_perm_matrix,
    plot_permutation_history_animation,
    restore_original_weights,
)
from ccmm.utils.utils import (
    fuse_batch_norm_into_conv,
    get_interpolated_loss_acc_curves,
    l2_norm_models,
    linear_interpolate,
    load_model_from_info,
    map_model_seed_to_symbol,
    normalize_unit_norm,
    project_onto,
    save_factored_permutations,
    vector_to_state_dict,
)

In [ ]:
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
    }
)
sns.set_context("talk")

cmap_name = "coolwarm_r"

from ccmm.utils.plot import Palette

palette = Palette(f"{PROJECT_ROOT}/misc/palette2.json")
palette

In [ ]:
logging.getLogger("lightning.pytorch").setLevel(logging.WARNING)
logging.getLogger("torch").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
pylogger = logging.getLogger(__name__)

## Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="matching")

In [ ]:
cfg = compose(config_name="matching", overrides=["model=vit", "dataset=cifar10"])

In [ ]:
core_cfg = cfg  # NOQA
cfg = cfg.matching

seed_index_everything(cfg)

## Hyperparameters

In [ ]:
num_test_samples = 5000
num_train_samples = 5000

## Load dataset

In [ ]:
transform = instantiate(core_cfg.dataset.test.transform)

train_dataset = instantiate(core_cfg.dataset.train, transform=transform)
test_dataset = instantiate(core_cfg.dataset.test, transform=transform)

train_subset = Subset(train_dataset, list(range(num_train_samples)))
train_loader = DataLoader(train_subset, batch_size=5000, num_workers=cfg.num_workers)

test_subset = Subset(test_dataset, list(range(num_test_samples)))

test_loader = DataLoader(test_subset, batch_size=1000, num_workers=cfg.num_workers)

In [ ]:
trainer = instantiate(cfg.trainer, enable_progress_bar=False, enable_model_summary=False)

## Load models

In [ ]:
from ccmm.utils.utils import load_model_from_artifact

run = wandb.init(project=core_cfg.core.project_name, entity=core_cfg.core.entity, job_type="matching")

# {a: 1, b: 2, c: 3, ..}
symbols_to_seed: Dict[int, str] = {map_model_seed_to_symbol(seed): seed for seed in cfg.model_seeds}

artifact_path = (
    lambda seed: f"{core_cfg.core.entity}/{core_cfg.core.project_name}/{core_cfg.dataset.name}_{core_cfg.model.model_identifier}_{seed}:latest"
)

# {a: model_a, b: model_b, c: model_c, ..}
models: Dict[str, LightningModule] = {
    map_model_seed_to_symbol(seed): load_model_from_artifact(run, artifact_path(seed)) for seed in cfg.model_seeds
}
model_orig_weights = {symbol: copy.deepcopy(model.model.state_dict()) for symbol, model in models.items()}

num_models = len(models)

pylogger.info(f"Using {num_models} models with architecture {core_cfg.model.model_identifier}")

## Load permutation specification

In [ ]:
from ccmm.matching.permutation_spec import PermutationSpec, PermutationSpecBuilder


class ViTPermutationSpecBuilder(PermutationSpecBuilder):
    def __init__(self, depth) -> None:
        self.depth = depth

    def create_permutation(self) -> PermutationSpec:

        axes_to_perm = {
            # layer norm
            "to_patch_embedding.to_patch_tokens.1.weight": (None,),  # (3*c*16)
            "to_patch_embedding.to_patch_tokens.1.bias": (None,),  # (3*c*16)
            # linear
            "to_patch_embedding.to_patch_tokens.2.weight": ("P_in", None),  # (dim, 3*c*16)
            "to_patch_embedding.to_patch_tokens.2.bias": ("P_in",),  # dim
            "pos_embedding": (None, None, "P_in"),  # (1, p+1, dim) probably P_transf_in or its own P
            "cls_token": (None, None, "P_in"),  # (1, 1, dim) probably P_transf_in or its own P
            **transformer_block_axes(self.depth, p_in="P_in", p_out="P_last"),
            # layer norm
            "mlp_head.0.weight": ("P_last",),  # (dim, )
            "mlp_head.0.bias": ("P_last",),  # (dim,)
            # linear
            "mlp_head.1.bias": (None,),  # (num_classes)
            "mlp_head.1.weight": (None, "P_last"),  # (num_classes, dim)
        }

        return self.permutation_spec_from_axes_to_perm(axes_to_perm)


def transformer_block_axes(depth, p_in, p_out):

    all_axes = {}

    for block_ind in range(depth):

        block_out = p_out if block_ind == depth - 1 else f"P{block_ind}_out"
        block_in = p_in if block_ind == 0 else f"P{block_ind-1}_out"

        block_axes = {
            # Attention
            ## layer norm
            f"transformer.layers.{block_ind}.0.norm.weight": (block_in,),  # (dim,) OK
            f"transformer.layers.{block_ind}.0.norm.bias": (block_in,),  # (dim,) OK
            f"transformer.layers.{block_ind}.0.temperature": (None,),  # (,)
            # HEADS
            f"transformer.layers.{block_ind}.0.to_q.weight": (
                f"P{block_ind}_attn_QK",
                block_in,
            ),  # (head_dim, dim) row, col OK
            f"transformer.layers.{block_ind}.0.to_k.weight": (
                f"P{block_ind}_attn_QK",
                block_in,
            ),  # (head_dim, dim) row, col OK
            f"transformer.layers.{block_ind}.0.to_v.weight": (None, block_in),  # (head_dim, dim) row, col OK
            # Attention out
            f"transformer.layers.{block_ind}.0.to_out.0.weight": (
                # f"{block_ind}_attn_out",
                None,
                # f"P{block_ind}_attn_V",
                None,
            ),  # (dim, dim)
            f"transformer.layers.{block_ind}.0.to_out.0.bias": (None,),  # (f"{block_ind}_attn_out",),  # (dim,)
            # shortcut
            # TODO: the rows perm of this one should be the product of block_in and {block_ind}_attn_out?
            f"transformer.layers.{block_ind}.1.identity": (block_in, None),  # (dim, dim) # WORKS
            # MLP
            ## layer norm
            f"transformer.layers.{block_ind}.2.net.0.weight": (None,),  # (f"{block_ind}_attn_out",),  # (dim, )
            f"transformer.layers.{block_ind}.2.net.0.bias": (None,),  # (f"{block_ind}_attn_out",),  # (dim,)
            #
            ## linear 1
            f"transformer.layers.{block_ind}.2.net.1.weight": (
                f"P{block_ind}_mlp_out",
                None,
                # f"{block_ind}_attn_out",
            ),  # (mlp_dim, dim)
            f"transformer.layers.{block_ind}.2.net.1.bias": (f"P{block_ind}_mlp_out",),  # (mlp_dim,)
            #
            ## linear 2
            f"transformer.layers.{block_ind}.2.net.4.weight": (
                block_out,
                f"P{block_ind}_mlp_out",
            ),  # (dim, mlp_dim) # WORKS
            f"transformer.layers.{block_ind}.2.net.4.bias": (block_out,),  # (dim,) # WORKS
            #
            # shortcut 2
            f"transformer.layers.{block_ind}.3.identity": (None, block_out),  # (dim, dim) # WORKS
        }

        all_axes.update(block_axes)

    return all_axes

In [ ]:
permutation_spec_builder = ViTPermutationSpecBuilder(core_cfg.model.depth)
permutation_spec = permutation_spec_builder.create_permutation()

ref_model = list(models.values())[0]
assert set(permutation_spec.layer_and_axes_to_perm.keys()) == set(ref_model.model.state_dict().keys())

## Git Re-Basin

In [ ]:
# always permute the model having larger character order, i.e. c -> b, b -> a and so on ...
from ccmm.matching.matcher import GitRebasinMatcher
from ccmm.matching.utils import get_inverse_permutations

symbols = set(symbols_to_seed.keys())
sorted_symbols = sorted(symbols, reverse=False)
fixed_symbol, permutee_symbol = "a", "b"
fixed_model, permutee_model = models[fixed_symbol].cpu(), models[permutee_symbol].cpu()

In [ ]:
# dicts for permutations and permuted params, D[a][b] refers to the permutation/params to map b -> a
gitrebasin_permutations = {symb: {other_symb: None for other_symb in symbols.difference(symb)} for symb in symbols}

matcher = GitRebasinMatcher(name="git_rebasin", permutation_spec=permutation_spec)
gitrebasin_permutations[fixed_symbol][permutee_symbol], perm_history = matcher(
    fixed=fixed_model.model, permutee=permutee_model.model
)

gitrebasin_permutations[permutee_symbol][fixed_symbol] = get_inverse_permutations(
    gitrebasin_permutations[fixed_symbol][permutee_symbol]
)

In [ ]:
from scripts.evaluate_matched_models import evaluate_pair_of_models

restore_original_weights(models, model_orig_weights)

updated_params = {fixed_symbol: {permutee_symbol: None}}

pylogger.info(f"Permuting model {permutee_symbol} into {fixed_symbol}.")

# perms[a, b] maps b -> a
updated_params[fixed_symbol][permutee_symbol] = apply_permutation_to_statedict(
    permutation_spec, gitrebasin_permutations[fixed_symbol][permutee_symbol], models[permutee_symbol].model.state_dict()
)
restore_original_weights(models, model_orig_weights)

lambdas = [0.5, 1.0]

gitrebasin_results = evaluate_pair_of_models(
    models,
    fixed_symbol,
    permutee_symbol,
    updated_params,
    train_loader,
    test_loader,
    lambdas,
    core_cfg,
)

In [ ]:
gitrebasin_results["test_loss_barrier"]

## Frank-Wolfe

In [ ]:
from ccmm.matching.frank_wolfe_matching import collect_perm_sizes, frank_wolfe_weight_matching_trial
from ccmm.matching.matcher import FrankWolfeMatcher

params_a = fixed_model.model.state_dict()
params_b = permutee_model.model.state_dict()
perm_sizes = collect_perm_sizes(permutation_spec, params_a)

initialization_method = "identity"

In [ ]:
perm_matrices, perm_matrices_history, new_obj, all_step_sizes = frank_wolfe_weight_matching_trial(
    params_a,
    params_b,
    perm_sizes,
    initialization_method,
    permutation_spec,
    200,
    device="cuda",
    return_step_sizes=True,
    global_step_size=True,
)

restore_original_weights(models, model_orig_weights)

In [ ]:
plt.plot([step_size for ind, step_size in enumerate(all_step_sizes)], color=palette["light red"])

plt.xlabel("Iteration")
plt.ylabel("Step size")
plt.title("Step size")

plt.savefig("figures/convergence_step_sizes.pdf", bbox_inches="tight")

In [ ]:
from ccmm.matching.weight_matching import solve_linear_assignment_problem

fw_permutations = {symb: {other_symb: None for other_symb in symbols.difference(symb)} for symb in symbols}
fw_permutations[fixed_symbol][permutee_symbol] = {
    p: solve_linear_assignment_problem(perm) for p, perm in perm_matrices.items()
}

In [ ]:
updated_params = {fixed_symbol: {permutee_symbol: None}}

In [ ]:
from scripts.evaluate_matched_models import evaluate_pair_of_models

restore_original_weights(models, model_orig_weights)

pylogger.info(f"Permuting model {permutee_symbol} into {fixed_symbol}.")

# perms[a, b] maps b -> a
updated_params[fixed_symbol][permutee_symbol] = apply_permutation_to_statedict(
    permutation_spec, fw_permutations[fixed_symbol][permutee_symbol], models[permutee_symbol].model.state_dict()
)
restore_original_weights(models, model_orig_weights)

lambdas = [0.0, 0.5, 1]  # np.linspace(0, 1, num=4)

fw_results = evaluate_pair_of_models(
    models,
    fixed_symbol,
    permutee_symbol,
    updated_params,
    train_loader,
    test_loader,
    lambdas,
    core_cfg,
)

In [ ]:
fw_results["test_loss_barrier"]